In [1]:
import pandas as pd
from pandas import json_normalize
import google.generativeai as genai
from google.api_core import retry
from sklearn.model_selection import train_test_split

import dotenv
import os

In [2]:
# Turn json into pandas dataframe
data = pd.read_json('large_eve.json', lines=True)
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


In [3]:
print("Records amount: ", len(data))

Records amount:  746909


In [4]:
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
0,2017-07-22T17:33:15.784100-0500,2.835707e+14,18519.0,alert,130.0,192.168.203.200,2328.0,192.181.145.109,445.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2017-07-22T17:33:16.571273-0500,1.519048e+15,21531.0,ssh,140.0,192.168.204.60,62990.0,192.168.26.254,22.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2017-07-22T17:33:17.561063-0500,9.632021e+14,28731.0,http,140.0,192.168.204.59,38011.0,192.168.24.201,80.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2017-07-22T17:33:17.595416-0500,9.632021e+14,28994.0,fileinfo,140.0,192.168.24.201,80.0,192.168.204.59,38011.0,TCP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
746904,2017-07-22T19:47:49.301548-0500,1.970284e+15,NaN,flow,NaN,192.168.203.200,2007.0,192.183.45.157,445.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746905,2017-07-22T19:47:49.301548-0500,2.251792e+15,NaN,flow,NaN,192.168.198.57,52498.0,192.168.21.141,22.0,TCP,...,NaN,"{'pkts_toserver': 2, 'pkts_toclient': 0, 'byte...",NaN,NaN,"{'tcp_flags': '02', 'tcp_flags_ts': '02', 'tcp...",NaN,NaN,NaN,NaN,NaN
746906,2017-07-22T19:47:49.301548-0500,8.444188e+14,NaN,flow,NaN,192.168.201.72,57805.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN
746907,2017-07-22T19:47:49.301548-0500,2.814688e+14,NaN,flow,NaN,192.168.201.72,43004.0,192.168.26.225,80.0,TCP,...,NaN,"{'pkts_toserver': 6, 'pkts_toclient': 6, 'byte...",NaN,NaN,"{'tcp_flags': '1b', 'tcp_flags_ts': '1b', 'tcp...",NaN,NaN,NaN,NaN,NaN


## Part 1

Get all DNS records

In [5]:
# Filter only dns events
data = data[data['event_type'] == "dns"]

In [6]:
len(data)

15749

In [7]:
data.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,vars,flow,icmp_type,icmp_code,tcp,smtp,email,app_proto_tc,app_proto_ts,stats
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
38,2017-07-22T17:33:24.990320-0500,2.022925e+15,54352.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,38051.0,2001:0500:0003:0000:0000:0000:0000:0042,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
39,2017-07-22T17:33:27.379891-0500,5.785448e+14,54519.0,dns,150.0,192.168.205.170,31393.0,192.168.207.4,53.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
41,2017-07-22T17:33:27.380146-0500,5.785448e+14,54520.0,dns,150.0,192.168.207.4,53.0,192.168.205.170,31393.0,UDP,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
data.shape

(15749, 28)

Filter all domains that are type A

In [9]:
# Normalize the json data inside dns column
dns_expanded = json_normalize(data['dns'])
# Join it with the original dataframe
data = data.join(dns_expanded, lsuffix='', rsuffix='_dns', how='right')
data.head()

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,app_proto_ts,stats,type,id,rrname,rrtype,tx_id_dns,rcode,ttl,rdata
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,answer,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,NaN,NaN,NXDOMAIN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,answer,54724,<root>,SOA,NaN,NXDOMAIN,20864.0,NaN


In [10]:
data.shape

(15749, 36)

In [11]:
# Filter only type A records
data = data[data['rrtype'] == 'A']
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,app_proto_ts,stats,type,id,rrname,rrtype,tx_id_dns,rcode,ttl,rdata
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,45082,api.wunderground.com,A,0.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15713,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,15886,secure.informaction.com.hsd1.pa.comcast.net,A,0.0,NaN,NaN,NaN
15716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN
15725,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,13068,wpad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
15737,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,59436,client-software.real.com,A,0.0,NaN,NaN,NaN


Domain filter to find only unique domains records

In [12]:
# Find only records with unique domain names
data = data.drop_duplicates(subset=['rrname'])
data

,timestamp,flow_id,pcap_cnt,event_type,vlan,src_ip,src_port,dest_ip,dest_port,proto,...,app_proto_ts,stats,type,id,rrname,rrtype,tx_id_dns,rcode,ttl,rdata
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,15529,api.wunderground.com,A,0.0,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,58278,stork79.dropbox.com,A,0.0,NaN,NaN,NaN
2,2017-07-22T17:33:16.661646-0500,1.327836e+15,22269.0,dns,110.0,2001:0dbb:0c18:0011:0260:6eff:fe30:0863,59680.0,2001:0500:0001:0000:0000:0000:803f:0235,53.0,UDP,...,NaN,NaN,query,54724,hpca-tier2.office.aol.com.ad.aol.aoltw.net,A,0.0,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,7425,safebrowsing.clients.google.com.home,A,0.0,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,12854,fxfeeds.mozilla.com,A,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15632,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,10109,ntp.ubuntu.com.localdomain,A,0.0,NaN,NaN,NaN
15661,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,46237,redir.metaservices.microsoft.com,A,0.0,NaN,NaN,NaN
15701,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,54310,ocsp.verisign.com,A,0.0,NaN,NaN,NaN
15716,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,query,33079,192.168.22.201:.stayonline.net,A,0.0,NaN,NaN,NaN


In [13]:
len(data)

177

Extract the Top Level Domain (TLD) from eath domain

In [14]:
def extract_tld(url):
    """
    Extract the Top-Level Domain (TLD) from a URL or domain name.
    
    Args:
        url (str): The URL or domain name to extract TLD from
        
    Returns:
        str: The extracted TLD or None if extraction fails
    """
    import re
    from urllib.parse import urlparse
    
    # Handle None or empty values
    if not url or not isinstance(url, str):
        return None
    
    # If the URL doesn't start with http/https, add a dummy scheme
    if not url.startswith(('http://', 'https://')):
        url = 'http://' + url
    
    try:
        # Parse the URL
        parsed_url = urlparse(url)
        
        # Extract the hostname
        hostname = parsed_url.netloc
        
        # Remove port number if present
        hostname = hostname.split(':')[0]
        
        # Extract domain parts
        domain_parts = hostname.split('.')
        
        # Handle special cases and ccTLDs with secondary domains (co.uk, com.au, etc.)
        special_tlds = ['co.uk', 'com.au', 'co.nz', 'org.uk', 'net.au', 'org.au', 
                        'ac.uk', 'gov.uk', 'gov.au', 'edu.au']
        
        if len(domain_parts) >= 2:
            possible_special = '.'.join(domain_parts[-2:])
            if possible_special in special_tlds and len(domain_parts) >= 3:
                return possible_special
            else:
                # Return just the TLD part (last part)
                return domain_parts[-1]
        else:
            return None
            
    except Exception as e:
        # Handle any parsing errors
        print(f"Error extracting TLD: {e}")
        return None

In [15]:
import tldextract

print(tldextract.extract('api.wunderground.com'))

print(tldextract.extract('safebrowsing.clients.google.com.home'))

ExtractResult(subdomain='api', domain='wunderground', suffix='com', is_private=False)
ExtractResult(subdomain='safebrowsing.clients.google.com', domain='home', suffix='', is_private=False)


In [16]:
data['domain_tld'] = data['rrname'].apply(tldextract.extract).apply(lambda x: x.domain+'.'+x.suffix).drop_duplicates()

/var/folders/wx/3fwd1ycn21q0f_72nz8kjrr00000gn/T/ipykernel_61868/2559329416.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['domain_tld'] = data['rrname'].apply(tldextract.extract).apply(lambda x: x.domain+'.'+x.suffix).drop_duplicates()


In [17]:
domain_list = data["domain_tld"].dropna().astype(str).drop_duplicates().tolist()
print(len(domain_list))
print(domain_list)

102
['wunderground.com', 'dropbox.com', 'aoltw.net', 'home.', 'mozilla.com', 'metasploit.com', 'aol.com', 'google.com', 'stayonline.net', '110phpmyadmin.', 'informaction.com', 'localdomain.', 'vmware.com', 'windows.com', 'ntkrnlpa.info', 'portswigger.net', 'offensive-security.com', 'stopbadware.org', 'arrancar.org', 'sql-ledger.org', 'backtrack-linux.org', 'theanime.cn', 'vpn.', 'phpmyadmin.net', 'microsoft.com', 'comcast.net', 'ubuntu.com', 'alexa.com', 'postgresql.org', 'sourceforge.net', 'freepbx.org', 'bigflickrfeed.com', 'gnu.org', 'wpad.', 'lan.', '56".', 'acunetix.com', 'windowsupdate.com', 'flickr.com', 'malwarecity.com', 'facebook.com', 'securityfocus.com', 'xmarks.com', '0.', 'cakephp.org', 'hec.net', 'adams.net', 'uidaho.edu', 'pdx.edu', 'clarkson.edu', 'rackspace.com', 'ecvps.com', 'wisc.edu', 'facebook.net', 'easynews.com', 'bluehost.com', 'netriplex.com', 'stanford.edu', 'tummy.com', 'fastserv.com', 'kernel.org', 'hmc.edu', 'liquidweb.com', '5ninesolutions.com', 'interwor

## Part 2

In [18]:
dotenv.load_dotenv()

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
genai.configure(api_key=GOOGLE_API_KEY)

In [19]:
retry_policy = {
    "retry": retry.Retry(predicate=retry.if_transient_error, initial=10, multiplier=1.5, timeout=200)
}

In [20]:
import enum

class Classification(enum.Enum):
    POSITIVE = "1"
    NEGATIVE = "0"

model = genai.GenerativeModel(
    'models/gemini-2.0-flash',
    generation_config=genai.GenerationConfig(
        temperature=0.95,
        top_p=0.95,
        top_k=80,
        response_mime_type="text/x.enum",
        response_schema=Classification
    ))

In [33]:
responses = dict()

In [34]:
for i in range(0, len(domain_list), 10):
    batch = domain_list[i:i+10]
    for index, domain in enumerate(batch):
        prompt = f"""Classify the following domain TLD as POSITIVE (algorithm generated) or NEGATIVE (legitimate):

        TLD to classify: {domain}

        Classification guidelines:
        - LEGITIMATE: Standard recognized TLDs (com, org, net, edu, gov, io, co, app, etc.)
        - DGA-GENERATED: Likely algorithmically generated TLDs showing these characteristics:
        * Random-appearing letter/number combinations
        * Unusual length (extremely short or long)
        * Non-pronounceable character sequences
        * Excessive digits mixed with letters

        Examples:
        - Legitimate: com, org, net, edu, gov, io, co, app, info, biz
        - DGA-Generated: xyz123, a7b9, m5k2p, qw34rt, 5j8k2
        """
        response = model.generate_content(prompt, request_options=retry_policy)
        print(response.text)
        responses[domain] = response.text

1
0
1
0
0
0
0
0
0
1
0
0
0
0
1
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
1
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0


In [36]:
print(responses)
print(len(responses))

{'wunderground.com': '1', 'dropbox.com': '0', 'aoltw.net': '1', 'home.': '0', 'mozilla.com': '0', 'metasploit.com': '0', 'aol.com': '0', 'google.com': '0', 'stayonline.net': '0', '110phpmyadmin.': '1', 'informaction.com': '0', 'localdomain.': '0', 'vmware.com': '0', 'windows.com': '0', 'ntkrnlpa.info': '1', 'portswigger.net': '0', 'offensive-security.com': '0', 'stopbadware.org': '0', 'arrancar.org': '0', 'sql-ledger.org': '0', 'backtrack-linux.org': '0', 'theanime.cn': '1', 'vpn.': '0', 'phpmyadmin.net': '0', 'microsoft.com': '0', 'comcast.net': '0', 'ubuntu.com': '0', 'alexa.com': '0', 'postgresql.org': '0', 'sourceforge.net': '0', 'freepbx.org': '0', 'bigflickrfeed.com': '0', 'gnu.org': '0', 'wpad.': '0', 'lan.': '0', '56".': '0', 'acunetix.com': '0', 'windowsupdate.com': '0', 'flickr.com': '0', 'malwarecity.com': '0', 'facebook.com': '0', 'securityfocus.com': '0', 'xmarks.com': '0', '0.': '0', 'cakephp.org': '0', 'hec.net': '0', 'adams.net': '0', 'uidaho.edu': '0', 'pdx.edu': '0', 

In [ ]:
for item in responses.items():
    print(item)

print(len(filtered_list))

0
